In [82]:
import pandas as pd
import numpy as np
import psycopg2 as p2
from psycopg2 import sql
from collections import Counter
from tqdm import tqdm
import pylcs
from rapidfuzz import fuzz

pd.set_option('display.width', 20000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 200)

In [2]:
dbname = "vulns_scanner"
user = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'

In [34]:
def get_df_from_bd(q):
    conn = p2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    cur = conn.cursor()
    cur.execute(q)
    colnames = [desc[0] for desc in cur.description]
    tuples = cur.fetchall()
    cur.close()
    df = pd.DataFrame(tuples, columns=colnames)
    return df

План:

1) Искать в БД юзая Левенштейн толкьо по продукту, а потом проверять вендора и версию?

2) Добавить эвристику для нормализации на длину запросов?

In [6]:
data = pd.read_excel('/home/mikhail/Downloads/processed_cpe_050425.xlsx')

In [181]:
product_ner = data['dedup_product'].apply(lambda x: x.lstrip('[\'').rstrip(']\'')).values
vendor_ner = data['dedup_vendor'].apply(lambda x: x.lstrip('[\'').rstrip(']\'')).values

In [5]:
res_df = pd.DataFrame()

for p in product_ner:
    q = f"""
    select initial_cpe, levenshtein(product, '{p}') as lev, length(product) as len
    from cpes c 
    order by "lev"
    limit 1000;
    """
    df = 

NameError: name 'product_name' is not defined

In [30]:
product_ner[:30].values

array(['', 'choicelistprovider', 'chetcpasswd', '', 'wp go maps', '',
       'zoom client for', '', '', 'scylladb', 'spip', 'pivx', 'v-server',
       'ltwcalendar', 'parcel', '', 'http server', '', 'dom4j',
       'wp matterport shortcode plugin', 'arm',
       'pleasant password server', 'jetspeed', '', 'matrix ios sdk',
       'evince', 'php-fusion', '', 'scaffold-helper', 'friendica'],
      dtype=object)

In [43]:
prodct = 'choicelistprovider'

In [47]:
df = get_df_from_bd(f"""
    select product, levenshtein(product, '{prodct}') as lev, length(product) as len_product, length('{prodct}') as len_query
    from cpes c 
    order by lev
    limit 1000;
    """)

In [48]:
df

,product,lev,len_product,len_query
0,contacts_provider,7,17,18
1,service_provider,8,16,18
2,ole_db_provider,8,15,18
3,service_provider,8,16,18
4,service_provider,8,16,18
...,...,...,...,...
995,cloud_server,11,12,18
996,mail_server,11,11,18
997,cobalt_strike,11,13,18
998,cobalt_strike,11,13,18


In [53]:
df_all = get_df_from_bd('select * from cpes limit 1000000;')

In [54]:
df_all.

,cpe_id_pk,cpe_version,part,vendor,product,version,update,edition,sw_edition,target_sw,target_hw,language,other,initial_cpe
0,498933,2.3,a,node-oojs_project,node-oojs,1.0.23,,,,,node.js,,,cpe:2.3:a:node-oojs_project:node-oojs:1.0.23:*:*:*:*:node.js:*:*
1,498934,2.3,a,node-oojs_project,node-oojs,1.0.24,,,,,node.js,,,cpe:2.3:a:node-oojs_project:node-oojs:1.0.24:*:*:*:*:node.js:*:*
2,498935,2.3,a,node-oojs_project,node-oojs,1.0.25,,,,,node.js,,,cpe:2.3:a:node-oojs_project:node-oojs:1.0.25:*:*:*:*:node.js:*:*
3,498936,2.3,a,node-oojs_project,node-oojs,1.0.26,,,,,node.js,,,cpe:2.3:a:node-oojs_project:node-oojs:1.0.26:*:*:*:*:node.js:*:*
4,498937,2.3,a,node-oojs_project,node-oojs,1.1.0,,,,,node.js,,,cpe:2.3:a:node-oojs_project:node-oojs:1.1.0:*:*:*:*:node.js:*:*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731541,498928,2.3,a,node-oojs_project,node-oojs,1.0.18,,,,,node.js,,,cpe:2.3:a:node-oojs_project:node-oojs:1.0.18:*:*:*:*:node.js:*:*
731542,498929,2.3,a,node-oojs_project,node-oojs,1.0.19,,,,,node.js,,,cpe:2.3:a:node-oojs_project:node-oojs:1.0.19:*:*:*:*:node.js:*:*
731543,498930,2.3,a,node-oojs_project,node-oojs,1.0.20,,,,,node.js,,,cpe:2.3:a:node-oojs_project:node-oojs:1.0.20:*:*:*:*:node.js:*:*
731544,498931,2.3,a,node-oojs_project,node-oojs,1.0.21,,,,,node.js,,,cpe:2.3:a:node-oojs_project:node-oojs:1.0.21:*:*:*:*:node.js:*:*


In [57]:
df_all['product'].nunique()

60672

In [ ]:
# def get_lcs_nonnorm(prod_name, unique_products):
#     lcs_scores = np.array(pylcs.lcs_of_list(prod_name, unique_products))
#     return unique_products[np.argmax(lcs_scores)], np.max(lcs_scores)

In [178]:
def get_lcs(ner_name, unique_entities):
    lcs_scores = np.array(pylcs.lcs_of_list(ner_name, unique_entities))
    if len(np.argwhere(lcs_scores == np.max(lcs_scores))) <= 1:
        return unique_entities[np.argmax(lcs_scores)], np.max(lcs_scores)
    else:
        candidates = unique_entities[np.argwhere(lcs_scores == np.max(lcs_scores))]
        len_of_query = len(ner_name)
        d = -1
        fit_cand = ''
        for cand in candidates:
            cand = cand[0]
            diff = abs(len_of_query - len(cand))
            # print(cand, d, diff)
            if d == -1:
                d = diff
                fit_cand = cand
            elif d > diff:
                d = diff
                fit_cand = cand
            else:
                continue
        return fit_cand, np.max(lcs_scores)


In [177]:
pylcs.lcs_of_list('chef automate', ['chef', 'change_and_configuration_management_database'])

[4, 11]

In [180]:
query_pr = [prodct]
unique_products = df_all['product'].unique()
unique_vendors = df_all['vendor'].unique()

In [ ]:
for pr in product_ner:
    if pr:
        print(f'Product NER: {pr}')
        # (prod, score)= get_lcs_nonnorm(pr, unique_products)
        # print('Old')
        # print(f'Found product in DB: {prod}')
        # print(f'score: {score}')
        # print('Normed')
        (prod, score)= get_lcs(pr, unique_products)
        print(f'Found product in DB: {prod}')
        print(f'score: {score}', end='\n\n')


Product NER: choicelistprovider
Old
Found product in DB: maven_artifact_choicelistprovider_\(nexus\)
score: 18
Normed
Found product in DB: maven_artifact_choicelistprovider_\(nexus\)
score: 18

Product NER: chetcpasswd
Old
Found product in DB: chetcpasswd
score: 11
Normed
Found product in DB: chetcpasswd
score: 11

Product NER: wp go maps
Old
Found product in DB: bwp-google-xml-sitemaps
score: 8
Normed
Found product in DB: wp_go_maps
score: 8

Product NER: zoom client for
Old
Found product in DB: zoom_client_for_meetings
score: 13
Normed
Found product in DB: zoom_client_for_meetings
score: 13

Product NER: scylladb
Old
Found product in DB: scylladb
score: 8
Normed
Found product in DB: scylladb
score: 8

Product NER: spip
Old
Found product in DB: spamsniper
score: 4
Normed
Found product in DB: spip
score: 4

Product NER: pivx
Old
Found product in DB: pivx
score: 4
Normed
Found product in DB: pivx
score: 4

Product NER: v-server
Old
Found product in DB: webpack-dev-server
score: 8
Normed

In [182]:
for pr in vendor_ner:
    if pr:
        print(f'Vendor NER: {pr}')
        (prod, score)= get_lcs(pr, unique_vendors)
        print(f'Found vendor in DB: {prod}')
        print(f'score: {score}', end='\n\n')


Vendor NER: apache
Found vendor in DB: apache
score: 6

Vendor NER: advantech
Found vendor in DB: advantech
score: 9

Vendor NER: snapdragon
Found vendor in DB: design_and_implementation_of_covid-19_directory_on_vaccination_system_project
score: 9

Vendor NER: jordy
Found vendor in DB: jordy_meow
score: 5

Vendor NER: apache
Found vendor in DB: apache
score: 6

Vendor NER: codewalkers
Found vendor in DB: codewalkers
score: 11

Vendor NER: cisco
Found vendor in DB: cisco
score: 5

Vendor NER: apache
Found vendor in DB: apache
score: 6

Vendor NER: intracom
Found vendor in DB: intracom
score: 8

Vendor NER: mikrotik
Found vendor in DB: mikrotik
score: 8

Vendor NER: gnome
Found vendor in DB: gnome
score: 5

Vendor NER: hp
Found vendor in DB: hp
score: 2

Vendor NER: x2engine
Found vendor in DB: x2engine
score: 8

Vendor NER: microsoft
Found vendor in DB: microsoft
score: 9

Vendor NER: flyte
Found vendor in DB: flyte
score: 5

Vendor NER: xinuos
Found vendor in DB: xinuos
score: 6

Vendo

Может быть несколько вендоров у одного продукта

In [172]:
df_all.groupby('product', as_index=False)['vendor'].nunique().sort_values('vendor', ascending=False).head(100)

,product,vendor
8955,cms,27
46088,server,19
19882,gallery,17
13151,desktop,15
21324,guestbook,15
24309,internet_security,15
28747,linux,14
9895,connect,13
46707,shopping_cart,13
2817,antivirus,12


In [176]:
df_all[df_all['product'].str.contains('chef')]

,cpe_id_pk,cpe_version,part,vendor,product,version,update,edition,sw_edition,target_sw,target_hw,language,other,initial_cpe
71383,570348,2.3,a,opscode,chef,0.8.12,,,,,,,,cpe:2.3:a:opscode:chef:0.8.12:*:*:*:*:*:*:*
71384,570349,2.3,a,opscode,chef,0.8.14,,,,,,,,cpe:2.3:a:opscode:chef:0.8.14:*:*:*:*:*:*:*
71385,570350,2.3,a,opscode,chef,0.8.16,,,,,,,,cpe:2.3:a:opscode:chef:0.8.16:*:*:*:*:*:*:*
71386,570351,2.3,a,opscode,chef,0.9.0,,,,,,,,cpe:2.3:a:opscode:chef:0.9.0:*:*:*:*:*:*:*
71387,570352,2.3,a,opscode,chef,0.9.2,,,,,,,,cpe:2.3:a:opscode:chef:0.9.2:*:*:*:*:*:*:*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566097,333330,2.3,a,progress,chef_infra_server,15.6.11,,,,,,,,cpe:2.3:a:progress:chef_infra_server:15.6.11:*:*:*:*:*:*:*
566098,333331,2.3,a,progress,chef_infra_server,15.6.12,,,,,,,,cpe:2.3:a:progress:chef_infra_server:15.6.12:*:*:*:*:*:*:*
597454,364682,2.3,a,jenkins,chef_identity,,,,,,jenkins,,,cpe:2.3:a:jenkins:chef_identity:*:*:*:*:*:jenkins:*:*
667596,434944,2.3,a,chef,chef_manage,1.11.4,,,,,,,,cpe:2.3:a:chef:chef_manage:1.11.4:*:*:*:*:*:*:*
